In [7]:
#checking 100 iter from cmd line train
import os
import torch
os.chdir('/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/src/vendor_run/mellotron')

from hparams import create_hparams
from model import Tacotron2
from text import cmudict, text_to_sequence
from train import warm_start_model
from loss_function import Tacotron2Loss
import waveglow.glow
from waveglow.denoiser import Denoiser
import waveglow
import sys
import IPython.display as ipd
from train import prepare_dataloaders
import time 
sys.path.append('waveglow/')
def to_arpabet(text):
    from g2p_en import G2p
    g2p = G2p()
    return " ".join(
        [
            f"{{ {s.strip()} }}" if s.strip() not in ",." else s.strip()
            for s in " ".join(g2p(text)).split("  ")
        ]
    )

def load_model(hparams):
    model = Tacotron2(hparams).cuda()
    if hparams.fp16_run:
        model.decoder.attention_layer.score_mask_value = finfo('float16').min

    return model


In [8]:

waveglow_path = '../../vendor_tools/waveglow/waveglow_256channels_universal_v4.pt'
waveglow_2 = torch.load(waveglow_path)['model'].cuda().eval()
denoiser = Denoiser(waveglow_2).cuda().eval()

In [78]:
hparams = create_hparams(None)
hparams.n_speakers = 1
hparams.training_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/lj_train.txt'
hparams.validation_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/lj_val.txt'
hparams.batch_size = 16
#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/lj_929_2/checkpoint_0'

hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/models/mellotron/vendor/mellotron_ljs.pt'
#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/lj_929/checkpoint_0'
#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/lj_929/checkpoint_200'
hparams.warm_start = True
learning_rate = 1e-3#
hparams.use_saved_learning_rate = True
#torch.manual_seed(hparams.seed)
#torch.manual_seed(1234)
torch.manual_seed(1234)
model = load_model(hparams)
# criterion = Tacotron2Loss()
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=hparams.weight_decay)
# optimizer.eps = 1e-16
model = warm_start_model(hparams.checkpoint_path, model, hparams.ignore_layers)
arpabet_dict = cmudict.CMUDict('data/cmu_dictionary')

Warm starting model from checkpoint '/mnt/disks/uberduck-experiments-v0/models/mellotron/vendor/mellotron_ljs.pt'


In [10]:
#     checkpoint_dict = torch.load(hparams.checkpoint_path, map_location='cpu')
#     model.load_state_dict(checkpoint_dict['state_dict'])
#     optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=hparams.weight_decay)
#     optimizer.load_state_dict(checkpoint_dict['optimizer'])
#     learning_rate = checkpoint_dict['learning_rate']

In [48]:
text = "its like this and like that and like this and uh"
text = to_arpabet(text)
text_encoded = torch.LongTensor(text_to_sequence(text, hparams.text_cleaners, arpabet_dict))[None, :].cuda()    
speaker_ids = torch.tensor([0], device='cuda:0').to(torch.int64)

xnew =(text_encoded, 0,speaker_ids,torch.zeros([1, 1, 481], device =  'cuda:0'))
with torch.no_grad():
    model.zero_grad()
    mel_outputs, mel_outputs_postnet, gate_outputs, rhythm = model.inference(xnew)
    audio = denoiser(waveglow_2.infer(mel_outputs_postnet, sigma=0.8), 0.01)[:, 0]
ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)

In [82]:
criterion = Tacotron2Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=hparams.weight_decay)
optimizer.eps = 1e-16

In [70]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=hparams.weight_decay)
optimizer.eps = 1e-16

In [75]:
#torch.manual_seed(1234)
#torch.manual_seed(5134964764772825327)
print(torch.seed(), torch.cuda.seed())
#torch.manual_seed(9991582768992915764)

3008216744530647353 None


In [79]:
#good_state = torch.get_rng_state()

In [83]:
torch.set_rng_state(good_state)

In [84]:
train_loader, valset, collate_fn, train_sampler = prepare_dataloaders(hparams)
timestart = time.time()
for epoch in range(1):
    for i, batch in enumerate(train_loader):
        
        x, y = model.parse_batch(batch)
        model.zero_grad()
        y_pred = model(x)
        loss = criterion(y_pred, y)
        print(i, loss)
        reduced_loss = loss.item()
        loss.backward()
        optimizer.step()
        timeend = time.time()
        print(epoch,i, timestart - timeend)
        timestart = timeend
        
        if i > 5:
            break

here
0 tensor(0.3886, device='cuda:0', grad_fn=<AddBackward0>)
0 0 -6.517996788024902
here
1 tensor(5.1483, device='cuda:0', grad_fn=<AddBackward0>)
0 1 -3.9166269302368164
here
2 tensor(1.5248, device='cuda:0', grad_fn=<AddBackward0>)


KeyboardInterrupt: 

In [16]:
text = "its like this and like that and like this and uh"
text = to_arpabet(text)
text_encoded = torch.LongTensor(text_to_sequence(text, hparams.text_cleaners, arpabet_dict))[None, :].cuda()    
speaker_ids = torch.tensor([0], device='cuda:0').to(torch.int64)

xnew =(text_encoded, 0,speaker_ids,torch.zeros([1, 1, 481], device =  'cuda:0'))
with torch.no_grad():
    model.zero_grad()
    mel_outputs, mel_outputs_postnet, gate_outputs, rhythm = model.inference(xnew)
    audio = denoiser(waveglow_2.infer(mel_outputs_postnet, sigma=0.8), 0.01)[:, 0]
ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)